In [1]:
# import libs
import cv2
import time
import threading
from pynput.keyboard import Controller
from ultralytics import YOLO
import mediapipe as mp
model = YOLO('models/glasses_v2_640.pt', task='detect')

In [2]:
keyboard = Controller()

# reduce gas percentage if driver is tired
REDUCED_GAS_PERCENTAGE = 0.7
FULL_GAS_PERCENTAGE = 1.0  
FULL_CYCLE_TIME = 1.0  

# global variables
stop_flag = False
blink_status = 'not_blinking'  # initial status
no_frame_detected_time = None

# define EAR (faster apply)
def calculate_ear(eye_landmarks):
    A = ((eye_landmarks[1][0] - eye_landmarks[5][0]) ** 2 + (eye_landmarks[1][1] - eye_landmarks[5][1]) ** 2) ** 0.5
    B = ((eye_landmarks[2][0] - eye_landmarks[4][0]) ** 2 + (eye_landmarks[2][1] - eye_landmarks[4][1]) ** 2) ** 0.5
    C = ((eye_landmarks[0][0] - eye_landmarks[3][0]) ** 2 + (eye_landmarks[0][1] - eye_landmarks[3][1]) ** 2) ** 0.5
    ear = (A + B) / (2.0 * C)
    return ear

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

face_mesh = mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

LEFT_EYE = [33, 160, 158, 133, 153, 144]  # P1=33, P2=160, P3=158, P4=133, P5=153, P6=144
RIGHT_EYE = [362, 385, 387, 263, 373, 380]  # P1=362, P2=385, P3=387, P4=263, P5=373, P6=380

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Fehler beim Öffnen der Kamera")
    exit()

# global variables for blink detection
detected_object = None
blink_count = 0
last_blink_time = time.time()
no_blink_start_time = time.time()
last_glasses_check_time = time.time()
glasses_check_interval = 5
treshhold = 0.15

# glasses detection
def detect_glasses(frame):
    results = model(frame, conf=0.4)
    detected_objects = results[0].boxes

    if len(detected_objects) == 0:
        return 'no_detection'
    else:
        closest_object = None
        max_area = 0

        for detected_object in detected_objects:
            box = detected_object.xyxy[0]
            area = (box[2] - box[0]) * (box[3] - box[1])
            if area > max_area:
                max_area = area
                closest_object = detected_object

        if closest_object:
            if closest_object.cls.item() == 0:
                return 'face_with_glasses'
            elif closest_object.cls.item() == 1:
                return 'face_without_glasses'
            else:
                return 'no_object'
        else:
            return 'no_object'

# blink detection
def detect_blinks(frame):
    global blink_count, last_blink_time, treshhold, blink_status

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_eye_landmarks = [(face_landmarks.landmark[idx].x * frame.shape[1], 
                                   face_landmarks.landmark[idx].y * frame.shape[0]) for idx in LEFT_EYE]
            right_eye_landmarks = [(face_landmarks.landmark[idx].x * frame.shape[1], 
                                    face_landmarks.landmark[idx].y * frame.shape[0]) for idx in RIGHT_EYE]

            left_ear = calculate_ear(left_eye_landmarks)
            right_ear = calculate_ear(right_eye_landmarks)
            ear = (left_ear + right_ear) / 2.0

            if ear < treshhold:
                cv2.putText(frame, "Blinking", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                blink_count += 1
                last_blink_time = time.time()

    if time.time() - last_blink_time > 10:
        cv2.putText(frame, "Not allowed to drive", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        blink_status = 'not_blinking'
    else:
        blink_status = 'blinking'

# process the frame with glasses detection and blink detection
def process_frame():
    global last_glasses_check_time, treshhold, no_frame_detected_time, stop_flag
    
    while not stop_flag:
        ret, frame = cap.read()
        if not ret:
            if no_frame_detected_time is None:
                no_frame_detected_time = time.time()
            elif time.time() - no_frame_detected_time > 20:
                print("Kein Frame erkannt, beende das Programm.")
                stop_flag = True
                break
            continue
        else:
            no_frame_detected_time = None

        detect_blinks(frame)
        
        current_time = time.time()
        if current_time - last_glasses_check_time > glasses_check_interval:
            print(f"Glasses Detection")
            last_glasses_check_time = current_time

            glasses_status = detect_glasses(frame)
            if glasses_status == 'face_with_glasses':
                treshhold = 0.05                            # lower treshhold for glasses --> simulating worse detection for people with glasses
            elif glasses_status == 'face_without_glasses':
                treshhold = 0.20                            # higher treshhold for no glasses --> simulating better detection for people without glasses
            else:
                treshhold = 0.20                            

        cv2.imshow('YOLOv8 Predictions and Eye Landmarks', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            stop_flag = True
            break

# reduce gas if driver is tired
def give_reduced_gas():
    global blink_status
    while not stop_flag:
        if blink_status == 'not_blinking':
            print("Müdigkeit erkannt, reduziere Gas.")
            keyboard.press(" ")
            time.sleep(REDUCED_GAS_PERCENTAGE * FULL_CYCLE_TIME)
            keyboard.release(" ")
            time.sleep((1 - REDUCED_GAS_PERCENTAGE) * FULL_CYCLE_TIME) 
        else:
            print("Keine Müdigkeit erkannt - Volles Gas.")
            time.sleep(1)

# start threads
processing_thread = threading.Thread(target=process_frame)
processing_thread.start()

gas_thread = threading.Thread(target=give_reduced_gas)
gas_thread.start()

processing_thread.join()
gas_thread.join()

cap.release()
cv2.destroyAllWindows()

Müdigkeit erkannt, reduziere Gas.


c:\Users\Leo\Desktop\AI\fairness_demonstator\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Glasses Detection
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.

Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
0: 480x640 1 face_without_glasses, 643.8ms
Speed: 6.0ms preprocess, 643.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Glasses Detection

Müdigkeit erkannt, reduziere Gas.
0: 480x640 1 face_without_glasses, 600.8ms
Speed: 3.5ms preprocess, 600.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)


c:\Users\Leo\Desktop\AI\fairness_demonstator\.venv\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Glasses Detection

Keine Müdigkeit erkannt - Volles Gas.
0: 480x640 1 face_without_glasses, 721.2ms
Speed: 3.5ms preprocess, 721.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Glasses Detection

Keine Müdigkeit erkannt - Volles Gas.
0: 480x640 1 face_without_glasses, 695.1ms
Speed: 5.0ms preprocess, 695.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Keine Müdigkeit erkannt - Volles Gas.
Glasses Detection

Müdigkeit erkannt, reduziere Gas.
0: 480x640 1 face_without_glasses, 725.8ms
Speed: 4.0ms preprocess, 725.8ms inference, 2.0ms po